In [8]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [9]:
!chmod 600 /root/.kaggle/kaggle.json

In [10]:
!kaggle datasets download -d danukhan/imdb-sentiment-dataset

Dataset URL: https://www.kaggle.com/datasets/danukhan/imdb-sentiment-dataset
License(s): unknown
 71% 36.0M/50.6M [00:00<00:00, 191MB/s] 
100% 50.6M/50.6M [00:00<00:00, 204MB/s]


In [11]:
import zipfile
zip_ref = zipfile.ZipFile('/content/imdb-sentiment-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [2]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
import pandas as pd
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
import pickle

In [3]:
ds_train = tf. data.TextLineDataset ("imdb.csv")
ds_test = tf . data. TextLineDataset ("imdb.csv")

In [4]:
for line in ds_train.skip(1) .take (5): #skip first row that contain name
  print (tf.strings.split(line, ',', maxsplit=4)) #maxsplit so that review want get split by comma

tf.Tensor(
[b'0' b'test' b'neg' b'0_2.txt'
 b'"Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner\'s character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he\'s better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher\'s ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in."'], shape=(5,), dtype=string)
tf.Tensor(
[b'1' b'

In [5]:
def filter_train(line):

  split_line = tf.strings.split(line, ",", maxsplit=4)
  dataset_belonging = split_line[1] # train, test
  sentiment_category = split_line [2] # pos, neg, unsup

  return (
      True
      if dataset_belonging == 'train' and sentiment_category != 'Unsup'
      else False
  )

def filter_test(line):

  split_line = tf.strings.split(line, ",", maxsplit=4)
  dataset_belonging = split_line[1] # train, test
  sentiment_category = split_line [2] # pos, neg, unsup

  return (
      True
      if dataset_belonging == 'test' and sentiment_category != 'Unsup'
      else False
  )

In [6]:
ds_train = tf. data.TextLineDataset ("imdb.csv").filter(filter_train)
ds_test = tf . data. TextLineDataset ("imdb.csv").filter(filter_test)

In [7]:
for line in ds_train.skip(1) .take (5): #skip first row that contain name
  print (tf.strings.split(line, ',', maxsplit=4))

tf.Tensor(
[b'25001' b'train' b'neg' b'10000_4.txt'
 b'"Airport \'77 starts as a brand new luxury 747 plane is loaded up with valuable paintings & such belonging to rich businessman Philip Stevens (James Stewart) who is flying them & a bunch of VIP\'s to his estate in preparation of it being opened to the public as a museum, also on board is Stevens daughter Julie (Kathleen Quinlan) & her son. The luxury jetliner takes off as planned but mid-air the plane is hi-jacked by the co-pilot Chambers (Robert Foxworth) & his two accomplice\'s Banker (Monte Markham) & Wilson (Michael Pataki) who knock the passengers & crew out with sleeping gas, they plan to steal the valuable cargo & land on a disused plane strip on an isolated island but while making his descent Chambers almost hits an oil rig in the Ocean & loses control of the plane sending it crashing into the sea where it sinks to the bottom right bang in the middle of the Bermuda Triangle. With air in short supply, water leaking in & havi

In [8]:
# TODO:
# 1. Create vocabulary
# 2. Numericalize text str -> indices (TokenTextEncoder)
# 3. Pad the batches so we can send in to an RNN for example

In [ ]:
!pip install tensorflow_text

In [24]:
tokenizer = tfds.deprecated.text.Tokenizer()

In [25]:
def build_vocabulary (ds_train, threshold=200):
  """ Build a vocabulary """
  frequencies = {}
  vocabulary = set()
  vocabulary.update (["sostoken" ])
  vocabulary. update (["eostoken"])

  for line in ds_train.skip (1):
    split_line = tf.strings.split(line, ",", maxsplit=4)
    review = split_line[4]
    tokenized_text = tokenizer.tokenize(review.numpy().lower())

    for word in tokenized_text:
      if word not in frequencies:
        frequencies [word] = 1
      else:
        frequencies [word] += 1

      # if we've reached the threshold
      if frequencies [word] == threshold:
        vocabulary.update(tokenized_text)
  return vocabulary

In [26]:
vocabulary = build_vocabulary(ds_train)

In [27]:
# Build vocabulary and save it to vocabulary.obj
vocab_file = open ('vocabulary.obj','wb')
pickle. dump (vocabulary, vocab_file)

In [17]:
# Loading the vocabulary
#vocab_file = open ("vocabulary .obj","rb")
#vocabulary = pickle. load (vocab_file)

In [29]:
encoder = tfds.deprecated.text.TokenTextEncoder(
    list (vocabulary),
    oov_token="<UNK>",
    lowercase=True,
    tokenizer=tokenizer)

def my_encoder (text_tensor, label):
  encoded_text = encoder. encode (text_tensor.numpy())
  return encoded_text, label

In [30]:
def encode_map_fn (line):
  split_line = tf.strings .split(line, ",", maxsplit=4)
  label_str = split_line [2] # neg, pos
  review = "sostoken " + split_line [4] + " eostoken"
  label = 1 if label_str == "pos" else 0
  (encoded_text, label) = tf.py_function (
      my_encoder,
      inp=[review, label],
      Tout=(tf.int64, tf.int32),
  ) #for computation graph we need to do this
  encoded_text.set_shape ([None]) #for computation graph we need to do this, review length can vary
  label.set_shape([]) #as label will be a list

  return encoded_text, label

In [31]:
AUTOTUNE = tf.data. experimental.AUTOTUNE
ds_train = ds_train.map (encode_map_fn, num_parallel_calls=AUTOTUNE) . cache ()
ds_train = ds_train.shuffle (25000)
ds_train = ds_train.padded_batch (32, padded_shapes= ([None], ())) #padded shape [none] as we dont know lendth of largest in bact
#and () fir label

In [33]:
ds_test = ds_test.map (encode_map_fn)
ds_test = ds_test.padded_batch (32, padded_shapes=([None], ()))

In [35]:
model = keras. Sequential(
    [
        layers.Masking(mask_value=0),
        layers.Embedding(input_dim=len(vocabulary)+2, output_dim=32,),
        layers.GlobalAveragePooling1D(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1),
    ]
)

In [36]:
model. compile (
    loss = keras. losses. BinaryCrossentropy (from_logits=True),
    optimizer=keras.optimizers.Adam (3e-4, clipnorm=1), metrics=["accuracy"],
)
model.fit (ds_train, epochs=15, verbose=2)
model.evaluate(ds_test)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:877: UserWarning: Layer 'embedding_1' (of type Embedding) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


2344/2344 - 145s - 62ms/step - accuracy: 0.8333 - loss: 0.4006
Epoch 2/15


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


2344/2344 - 68s - 29ms/step - accuracy: 0.8333 - loss: 0.4026
Epoch 3/15
2344/2344 - 72s - 31ms/step - accuracy: 0.8334 - loss: 0.3945
Epoch 4/15
2344/2344 - 80s - 34ms/step - accuracy: 0.8335 - loss: 0.3818
Epoch 5/15
2344/2344 - 83s - 35ms/step - accuracy: 0.8402 - loss: 0.3529
Epoch 6/15
2344/2344 - 80s - 34ms/step - accuracy: 0.8515 - loss: 0.3309
Epoch 7/15
2344/2344 - 76s - 32ms/step - accuracy: 0.8594 - loss: 0.3167
Epoch 8/15
2344/2344 - 72s - 31ms/step - accuracy: 0.8661 - loss: 0.3032
Epoch 9/15
2344/2344 - 84s - 36ms/step - accuracy: 0.8702 - loss: 0.2935
Epoch 10/15
2344/2344 - 66s - 28ms/step - accuracy: 0.8749 - loss: 0.2852
Epoch 11/15
2344/2344 - 79s - 34ms/step - accuracy: 0.8781 - loss: 0.2753
Epoch 12/15
2344/2344 - 83s - 35ms/step - accuracy: 0.8856 - loss: 0.2629
Epoch 13/15
2344/2344 - 82s - 35ms/step - accuracy: 0.8871 - loss: 0.2567
Epoch 14/15
2344/2344 - 83s - 35ms/step - accuracy: 0.8904 - loss: 0.2526
Epoch 15/15
2344/2344 - 62s - 27ms/step - accuracy: 0.896

[1.3289943933486938, 0.5113999843597412]

In [37]:
#for large dataset that is in multiple files we can do

In [38]:
file_names = ['test_examplel.csv','test_example2.csv', 'test_example3.csv']

dataset = tf.data. TextLineDataset(file_names)

In [39]:
#or if want different preprocessing

dataset1 = tf .data. TextLineDataset ('test_example1.cs').skip (1)#.map (preprocess
dataset2 = tf .data. TextLineDataset ('test_example2csv') .skip (1)#.map (preprocess
dataset3 = tf .data. TextLineDataset ('test_example.csv') .skip (1)# .map (preprocess
dataset = dataset1. concatenate (dataset2). concatenate (dataset3)

In [40]:
#now if we have trasnlation task and two files contaning dataset for each lang then

tokenizer = tfds.deprecated.text.Tokenizer ()
english = tf .data. TextLineDataset ("english.csv")
swedish = tf .data. TextLineDataset ("swedish.csv")
dataset = tf .data. Dataset. zip ((english, swedish))

In [ ]:
for eng, swe in dataset.skip (1):
  print(tokenizer.tokenize (eng .numpy ()))
  print(tokenizer.tokenize (swe.numpy () . decode ("UTF-8"))) #we decode this as this lang contain special chars